In [1]:
%run init_notebookspace.py

DATA_DIR is existant under: C:\Users\LukasGrahl\Documents\GIT\memoire1\data


In [2]:
from settings import MODEL_DIR

In [3]:
import sympy as sp
import numpy as np

from src.filtering_sampling import solve_updated_mod

# A simple monetary model (Galì, 2008) Chapter 2

## derivation

#### The household
$$
    \max \mathbb{E}_t \left[ \sum_{s=0}^{\inf} \beta^{t+s} U(C_{t+s}, N_{t+s}) \right]
    \\ s.t.
    \\ P_t C_t + Q_t B_t = B_{t-1} + W_t N_t 
    \\ U(C_t, N_t) = \frac{C_t^{1 - \sigma}}{(1 - \sigma)} - \frac{N_t^{1 + \phi}}{(1 + \phi)}
    \\ Q_t = \frac{1}{1+i_t}
$$

    FOC: 
    the FOC provide labour supply and the Euler Equation

$$ 
    - \frac{U_{n,t}}{U_{c, t}} = \frac{W_t}{P_t} = C_t^\sigma N_t^\phi
    \\ Q_t = \beta \mathbb{E}_t \{
                                        \frac{U_{c, t+1}}{U_{c, t}} \frac{P_t}{P_{t+1}}
                                    \}
            = \beta \mathbb{E}_t \{
                                        \frac{C_{t+1}}{C_t}^{-\sigma} \frac{P_t}{P_{t+1}}
                                    \}
$$

#### The firm
$$
    \max Y_t = A_t N_t^{1-\alpha} 
    \\ s.t.
    \\ P_t Y_t = W_t N_t
$$

    FOC:
    the FOC provides labour demand

$$
    W_t = (1 - \alpha) A_t N_t^{-\alpha} P_t
$$


#### Log linearisation
##### The Euler Equation 
$$
    1 = \mathbb{E}_t \{
                        \exp(i_t + \sigma \Delta c_{t+1} - \pi_{t+1} - \rho)
                    \}
$$

In the steady state consumption grows at the constant rate $\gamma$. Moreover $\pi$ is constant, hence for the above to hold:
$$
    i_{ss} = \sigma \gamma + \pi_{ss} + \rho
$$


plugging this into a 1st order Taylor approximation: 
$$
    1 \approx 1 + (i_t - i_{ss}) - \sigma(\Delta c_{t+1} - \gamma) - (\pi_{t+1} - \pi_{ss})
    \\ 1 \approx i_t + \mathbb{E}_t \{
                                        \sigma \Delta c_{t+1} + \pi_{t+1}
                                        \} + \rho
$$


and thus the Euler equation can be written as:
$$
    c_t = \mathbb{E}_t \{ c_{t+1} \} - \frac{1}{\sigma} (i_t - \rho - \mathbb{E}_t \{ \pi_{t+1} \} )
$$
##### the production function
$$
    y_t = a_t + (1 - \alpha) n_t
$$
##### labour demand and supply
rewriting labour demand and supply in terms of $w_t$ allows to elimante wage and to derive an equilbrium equivalence of $a_t$ and $n_t$
$$
    w_t = a_t - \alpha n_t + log(1-\alpha) + p_t = \sigma c_t + \phi n_t + p_t
$$

#### Euquilibirum

Under equilibrium condition all $y_t$ will be consumed, wherefore $y_t = c_t$. Setting labour demand equal to supply and substituting $c_t$ for output yields two expresssion of labour and output in terms of technology level $a_t$: 
$$
    \bf n_t = a_t \psi_{n,a} + \zeta_{n}
    \\
    y_t = a_t \psi_{y, a} + \zeta_{n} (1-\alpha)
$$

where the $\psi$s capture the following parameters
$$
    \psi_{n, a} = \frac{1-\sigma}{phi + \sigma + (1-\sigma) \alpha}
    \\
    \zeta_n = \frac{log(1-\alpha)}{phi + \sigma + (1-\sigma) \alpha}
    \\
    \psi_{y, a} = 1 + \psi_{n, a} (1-\alpha)
$$
casting this into the Euler condition and replacing $c_t$ with output yields an expression for the real interest rate
$$
    r_t \equiv i_t - \mathbb{E}_t \{\pi_{t+1} \}
    \\
    y_t = \mathbb{E}_t \{ y_{t+1} \} - \frac{1}{\sigma} (r_t + \mathbb{E}_t \{\pi_{t+1} \} - \mathbb{E}_t \{\pi_{t+1} \} - rho)
    \\
$$
$$
    \bf
    r_t =  \rho + \sigma \mathbb{E}_t \{ \Delta y_{t+1} \} = \rho + \sigma \psi_{y, a} \mathbb{E}_t \{\Delta a_{t+1} \}
$$

To close the model an interest rate rule is required, for the purpose of the simple model I will be the below rule with $\phi_{\pi}$ assumed to be greater 1:

$$
    i_t = \rho + \phi_{\pi} \pi_t
    \\ \hat{r_t} = r - \rho = i_t + \mathbb{E}_t \{ \pi_{t+1} \} = \phi_{\pi} \pi_t - \mathbb{E}_t \{ \pi_{t+1} \}
$$

In [4]:
# smpy representation
sp_symbols = sp.symbols(
    ['alpha', 'sigma', 'phi', 'rho', 'a_t', 'da_t']
)
alpha, sigma, phi, rho, at, dat = sp_symbols

psi_na = (1 - sigma) / (phi + sigma + (1 - sigma) * alpha)
zeta_n = sp.log(1 - alpha) / (phi + sigma + (1 - sigma) * alpha)
psi_ya = 1 + psi_na * (1 - alpha)

nt = at * psi_na + zeta_n
yt = at * psi_ya + sp.log(1-alpha) * zeta_n
rt = rho + sigma * psi_ya * dat

lamb_nt = sp.lambdify([alpha, sigma, phi, rho, at, dat], nt)
lamb_yt = sp.lambdify([alpha, sigma, phi, rho, at, dat], yt)
lamb_rt = sp.lambdify([alpha, sigma, phi, rho, at, dat], rt)

In [5]:
from config import mod4_params
params = [mod4_params[item] for item in ['alpha', 'sigma_C', 'sigma_L', 'beta']]

# get rho from beta
params[-1] = - np.round(np.log(params[-1]), 4)
# append at and dat
params.extend([1, 0])

### steady state variables

In [6]:
# labour
lamb_nt(*params)

-0.2799347116067531

In [7]:
# output
lamb_yt(*params)

0.958067344600637

In [8]:
# real interest rate
lamb_rt(*params)

0.0101

### Mickey Mouse RBC

In [9]:
_ = sp.symbols(['c_t', 'c_{t+1}', 'r_t', 'r_{t+1}', 'w_t', 'l_t', 'y_t', 'a_t', 'a_{t+1}', 'i_t', 'k_t', 'k_t+1', 'mc_t'])
c, c1, r, r1, w, l, y, a, a1, i, k, k1, mc = _

cs, rs, ws, ls, ys, a_s, i_s, ks, mcs = sp.symbols([f'{item}^*' for item in ['c', 'r', 'w', 'l', 'y', 'a', 'i', 'k', 'mc']])

_ = sp.symbols(['alpha', 'beta', 'delta', 'phi', 'rho_a', 'epsilon_a', 'sigma'])
alpha, beta, delta, phi, rho_a, epsilon_a, sigma = _

In [10]:
eqs = {
    'euler': sp.Eq(c **(- sigma), beta * c1 **(- sigma) * (r1 + 1 - delta)),
    'lsupply': sp.Eq(l ** phi, c ** + sigma * w),
    'ldemand': sp.Eq((1 - alpha) * y / l, w),
    'kdemand': sp.Eq(alpha * y / k, r),
    'prod': sp.Eq(y, a * k ** alpha * l ** (1 - alpha)),
    'ksupply': sp.Eq(k1, i + (1 - delta) * k),
    'budget': sp.Eq(y, c + i),
    'foc_r': sp.Eq(r, mc * alpha * y * k),
    'foc_w': sp.Eq(w, mc * (1 - alpha) * y * 1 / l)
}

In [21]:
# costas
eqs = {
    'euler': sp.Eq((c1 / c) ** sigma, beta * ((1 - delta) + r1)),
    'lsupply': sp.Eq(c ** sigma * l ** phi, w),
    'ldemand': sp.Eq(l, (1 - alpha) * y / w),
    'kdemand': sp.Eq(k, alpha  * y / r),
    'prod': sp.Eq(y, a * k ** alpha * l ** (1 - alpha)),
    'ksupply': sp.Eq(i, k1 - (1 - delta) * k),
    'budget': sp.Eq(y, c + i),
    'tech': sp.Eq(sp.log(a1), (1 - rho_a) * sp.log(a_s) + rho_a * sp.log(a) + epsilon_a),
}

In [22]:
subs_dict = {c: cs, c1: cs, r: rs, r1: rs, w: ws, l: ls, y: ys, a: a_s, i: i_s, k: ks, k1: ks, mc: mcs}

eqs_ss = {k: item.subs(subs_dict) for k, item in eqs.items()}

In [23]:
r_ss = sp.solve(eqs_ss['euler'], rs)[0]
a_ss = 1
mc_ss = 1

In [24]:
# steady state

In [25]:
r_ss = sp.solve(eqs_ss['euler'], rs)[0]
c_ss = ((1 - alpha) * ys) ** (sigma / phi) * ws **((1 + sigma) / phi)
i_ss = alpha * delta * ys / rs
w_ss = (1 - alpha) * (alpha / r_ss) ** (alpha / (1 - alpha))

In [26]:
y_ss = (r_ss / (r_ss + alpha * delta)) ** (sigma / (sigma + phi)) * (1 - alpha) ** (- sigma / (sigma + phi)) * w_ss ** ((1 + sigma)/(sigma + phi))

In [30]:
sp.simplify(y_ss)

((1 - alpha)/(alpha*beta/(beta*(delta - 1) + 1))**(alpha/(alpha - 1)))**((sigma + 1)/(phi + sigma))*((beta*(delta - 1) + 1)/(beta*(alpha*delta + delta - 1) + 1))**(sigma/(phi + sigma))/(1 - alpha)**(sigma/(phi + sigma))

In [31]:
sp.simplify(eqs_ss['ldemand'].subs({ys: y_ss, ws: w_ss}))

Eq(l^*, ((1 - alpha)/(alpha*beta/(beta*(delta - 1) + 1))**(alpha/(alpha - 1)))**((sigma + 1)/(phi + sigma))*((beta*(delta - 1) + 1)/(beta*(alpha*delta + delta - 1) + 1))**(sigma/(phi + sigma))*(alpha*beta/(beta*(delta - 1) + 1))**(alpha/(alpha - 1))/(1 - alpha)**(sigma/(phi + sigma)))

In [535]:
sp.solve(y_ss, ys)

NotImplementedError: multiple generators [(-alpha*y^* + y^*)**(sigma/phi), y^*]
No algorithms are implemented to solve equation -alpha*delta*y^*/r^* - w^***((sigma + 1)/phi)*(y^**(1 - alpha))**(sigma/phi) + y^*

In [530]:
c_ss

w^***((sigma + 1)/phi)*(y^**(1 - alpha))**(sigma/phi)

# New Keynsian Model (Galì, 2008) Chapter 4

In [48]:
Y, pi, r, v, a = sp.symbols('Y \pi, r, v, a')
omega, sigma, beta, phi_pi, kappa, rho, psi_ya, phi_y = sp.symbols('omega sigma beta, \phi_{\pi} kappa rho \psi_{ya} \phi_{y}')

a = 0
v = 0

eq_1 = Y - (omega * (sigma * Y + (1 - beta) * phi_pi * pi) + omega * (r - v))
eq_2 = pi - (omega * (sigma * kappa * Y + (kappa + beta * ( sigma + phi_y ) ) * pi ) + omega * kappa * (r - v))
eq_3 = r - (rho + sigma * psi_ya * (a - a))

A, y = sp.linear_eq_to_matrix([eq_1, eq_2, eq_3], [Y, pi, r])

sp.Matrix([[A, y]]).rref()[0]

Matrix([
[1, 0, 0, (-\phi_{\pi}*beta*kappa*omega**2*rho + \phi_{\pi}*kappa*omega**2*rho - \phi_{y}*beta*omega**2*rho - beta*omega**2*rho*sigma - kappa*omega**2*rho + omega*rho)/(\phi_{\pi}*beta*kappa*omega**2*sigma - \phi_{\pi}*kappa*omega**2*sigma + \phi_{y}*beta*omega**2*sigma - \phi_{y}*beta*omega + beta*omega**2*sigma**2 - beta*omega*sigma + kappa*omega**2*sigma - kappa*omega - omega*sigma + 1)],
[0, 1, 0,                                                                                                                                               kappa*omega*rho/(\phi_{\pi}*beta*kappa*omega**2*sigma - \phi_{\pi}*kappa*omega**2*sigma + \phi_{y}*beta*omega**2*sigma - \phi_{y}*beta*omega + beta*omega**2*sigma**2 - beta*omega*sigma + kappa*omega**2*sigma - kappa*omega - omega*sigma + 1)],
[0, 0, 1,                                                                                                                                                                                                

In [5]:
Y, pi_q, i, s = sp.symbols('Y \pi_q, i, s')
alpha_m, rho_s, beta, kappa, lambda_p, Gamma_s, phi_pi = sp.symbols('alpha_m rho_{s} beta kappa lambda_{p} \Gamma_{s} \phi_{\pi}')

s = 1

eq_1 = Y - Y + (i - pi_q ) + alpha_m * (1 - rho_s) / (1 - alpha_m) * s 
eq_2 = pi_q - beta * pi_q + kappa * Y + lambda_p + Gamma_s * s
eq_3 = i - phi_pi * pi_q

A, y = sp.linear_eq_to_matrix([eq_1, eq_2, eq_3], [Y, pi_q, i])

x = sp.Matrix([[A, y]]).rref()[0]

In [6]:
x

Matrix([
[1, 0, 0, (-\Gamma_{s}*\phi_{\pi}*alpha_m + \Gamma_{s}*\phi_{\pi} + \Gamma_{s}*alpha_m - \Gamma_{s} - \phi_{\pi}*alpha_m*lambda_{p} + \phi_{\pi}*lambda_{p} - alpha_m*beta*rho_{s} + alpha_m*beta + alpha_m*lambda_{p} + alpha_m*rho_{s} - alpha_m - lambda_{p})/(\phi_{\pi}*alpha_m*kappa - \phi_{\pi}*kappa - alpha_m*kappa + kappa)],
[0, 1, 0,                                                                                                                                                                                                                                                 (-alpha_m*rho_{s} + alpha_m)/(\phi_{\pi}*alpha_m - \phi_{\pi} - alpha_m + 1)],
[0, 0, 1,                                                                                                                                                                                                                           (-\phi_{\pi}*alpha_m*rho_{s} + \phi_{\pi}*alpha_m)/(\phi_{\pi}*alpha_m - \phi_{\pi} - alpha_m + 1)]])

In [8]:
x[3]

(-\Gamma_{s}*\phi_{\pi}*alpha_m + \Gamma_{s}*\phi_{\pi} + \Gamma_{s}*alpha_m - \Gamma_{s} - \phi_{\pi}*alpha_m*lambda_{p} + \phi_{\pi}*lambda_{p} - alpha_m*beta*rho_{s} + alpha_m*beta + alpha_m*lambda_{p} + alpha_m*rho_{s} - alpha_m - lambda_{p})/(\phi_{\pi}*alpha_m*kappa - \phi_{\pi}*kappa - alpha_m*kappa + kappa)